* Реализовать несколько стратегий перебора (минимизации) $k$ (число использованных цветов)
* Побенчмаркать: солверы/стратегии
* Разобраться с тем, как солвер работает с assumption'ами, а также с тем, как pysat строит "ядро" по assumption-set'у


* Реализовать стратегии таким образом, чтобы солвер не создавался заново
* Выводить результаты в csv
* Проверить на данных из dimacs


* Реализовать стратегии спуска, используя информацию о том, во сколько цветов солвер раскрасил граф на прошлой итерации
* Реализовать bottom-up стратегию, задействуя "ядро"
* Вывести всякую дополнительную информацию по итерациям
* Переименовать inflate -> bottomup, shrink -> topdown

In [44]:
from pysat.formula import CNF, IDPool
from pysat.solvers import Cadical, Glucose3, Glucose4, Lingeling, MapleChrono
from pysat.solvers import MapleCM, Maplesat, Minicard, Minisat22, MinisatGH 
from pysat.card import CardEnc, EncType

from timeit import default_timer as timer
import os
import random as rand
import pandas as pd

vpool = IDPool(start_from=1)

def color(vertex, color):
    return vpool.id(f'color@{vertex}@{color}')

def color_usage(color):
    return vpool.id(f'color_usage@{color}')

def main_coloring(n, k, edges, use_color_restrictions=False):
    cnf = CNF()
    for v in range(n):
        for i in range(k):
            for j in range(i + 1, k):
                cnf.append([-color(v, i), -color(v, j)])
    for v in range(0, n):
        cnf.append([color(v, i) for i in range(k)])
    for a, b in edges:
        for i in range(k):
            cnf.append([-color(a, i), -color(b, i)])
    if not use_color_restrictions:
        return cnf
    for v in range(n):
        for i in range(1, k):
            cnf.append([-color(v, i), color_usage(i)])
    for i in range(1, k - 1):
        cnf.append([color_usage(i), -color_usage(i + 1)])
    cnf.append([color_usage(0)])
    return cnf


def predict_coloring(cnf, n, k, assumptions, solver):
    if solver.solve() if assumptions is None else solver.solve(assumptions):
        model = solver.get_model()
        result = {v for v in model if v > 0}
        assert(0 not in model)
        colors = {v: i for v in range(n) for i in range(k) if color(v, i) in result}
        n_colors = len({colors[v] for v in colors})
        return (n_colors, colors), solver.time()
    else:
        return None, solver.time()
            
def print_results(strategy_name, solver_name, graph, colors, times):
    print(f"strategy={strategy_name} solver={solver_name} graph={graph} time={times[0]}:")
    if colors is None:
        print("UNSAT")
        print(f"Core: {s.get_core()}\n\n")
    else:
        print(f"SAT")
        print(f"{colors}\n\n")
        
def naive_coloring(n):
    return n, {v: v for v in range(n)}

def stats_diff(prev_stats, cur_stats):
    return {s: cur_stats[s] - prev_stats[s] for s in ['restarts', 'conflicts', 'decisions', 'propagations']}
            
def naive_bottomup_strategy(n, edges, solver):
    times = list()
    stats = list()
    for i in range(1, n):
        cnf = main_coloring(n, i, edges) 
        with solver(bootstrap_with=cnf.clauses, use_timer=True) as s:
            colors, time = predict_coloring(cnf, n, i, None, s)
            times.append(time)
            stats.append(s.accum_stats())
            stats[-1]["n_wanted"] = i
            stats[-1]["n_found"] = colors[0] if colors is not None else "None"
            if colors is not None:
                return colors, (sum(times), times), stats
    return naive_coloring(n), (sum(times), times), stats

def naive_topdown_strategy(n, edges, solver):
    answer = naive_coloring(n)
    times = list()
    stats = list()
    while answer[0] - 1 > 0:
        cnf = main_coloring(n, answer[0] - 1, edges)
        with solver(bootstrap_with=cnf.clauses, use_timer=True) as s:
            colors, time = predict_coloring(cnf, n, answer[0] - 1, None, s)
            times.append(time)
            stats.append(s.accum_stats())
            stats[-1]["n_wanted"] = answer[0] - 1
            stats[-1]["n_found"] = colors[0] if colors is not None else "None"
            if colors is not None:
                answer = colors
            else:
                return answer, (sum(times), times), stats
    return answer, (sum(times), times), stats

def top_down_strategy_main(n, edges, solver, strategy):
    answer = naive_coloring(n)
    times = list()
    stats = list()
    cnf = main_coloring(n, answer[0] - 1, edges, True)
    with solver(bootstrap_with=cnf.clauses, use_timer=True) as s:
        while answer[0] - 1 > 0:
            colors, time = strategy(cnf, answer[0] - 1, s)
            times.append(time)
            stats.append(s.accum_stats() if len(stats) == 0 else stats_diff(stats[-1], s.accum_stats()))
            stats[-1]["n_wanted"] = answer[0] - 1
            stats[-1]["n_found"] = colors[0] if colors is not None else "None"
            if colors is not None:
                answer = colors
            else:
                return answer, (s.time_accum(), times), stats
        return answer, (s.time_accum(), times), stats
    
def core_strategy(n, edges, solver):
    times = list()
    stats = list()
    cnf = main_coloring(n, n - 1, edges, True)
    n_colors = 2
    with solver(bootstrap_with=cnf.clauses, use_timer=True) as s:
        while True:
            colors, time = predict_coloring(cnf, n, n - 1, [-color_usage(j) for j in reversed(range(n_colors, n - 1))], s)
            times.append(time)
            stats.append(s.accum_stats() if len(stats) == 0 else stats_diff(stats[-1], s.accum_stats()))
            stats[-1]["n_wanted"] = n_colors
            stats[-1]["n_found"] = colors[0] if colors is not None else "None"
            if colors is not None:
                return colors, (s.time_accum(), times), stats
            else:
                n_colors = min([i for i in range(n_colors, n - 1) if -color_usage(i) in set(s.get_core())]) + 1
        return naive_coloring(n), (s.time_accum(), times), stats
    
def assumptions_strategy(n, edges, solver):
    return top_down_strategy_main(n, edges, solver, 
                                  lambda cnf, i, s : predict_coloring(cnf, n, n - 1, [-color_usage(j) for j in range(i, n - 1)], s))

def single_assumption_strategy(n, edges, solver):
    return top_down_strategy_main(n, edges, solver, 
                                  lambda cnf, i, s : predict_coloring(cnf, n, n - 1, [-color_usage(i)] if i != n - 1 else None, s))

def clauses_strategy(n, edges, solver):
    def internal(cnf, i, s):
        s.add_clause([-color_usage(i)])
        return predict_coloring(cnf, n, n - 1, None, s)
    return top_down_strategy_main(n, edges, solver, internal)

def read_graph(file):
    n = None
    edges = list()
    with open(os.path.join('graphs', file),'r') as f:
        content = [s.split() for s in f.readlines()]
        for s in content:
            if s[0] == 'c':
                continue
            elif s[0] == 'p':
                n = int(s[2])
            elif s[0] == 'e':
                edges.append((int(s[1]) - 1, int(s[2]) - 1))
            else:
                assert(False)
    assert(n is not None)
    return n, edges

def read_graphs():
    return sorted([(file, read_graph(file)) for file in os.listdir('graphs') 
            if os.path.isfile(os.path.join('graphs', file))], key=lambda item : item[1][0])

def generate_graph(n, es):
    edges = set()
    for i in range(es):
        a = rand.randint(0, n - 1)
        b = rand.randint(0, n - 1)
        a, b = min(a, b), max(a, b)
        if a != b and (a, b) not in edges:
            edges.add((a, b))
    return n, list(edges)

def concat_all(lists):
    res = list()
    for l in lists:
        res = res + l
    return res

def solutions2csv(solutions):
    max_it = max([len(solutions[i][3][1]) for i in range(len(solutions))])
    sets = [("Id"            , [i + 1              for i in range(len(solutions))]),
            ("Graph"         , [solutions[i][0]    for i in range(len(solutions))]),
            ("Solver"        , [solutions[i][1]    for i in range(len(solutions))]),
            ("Strategy"      , [solutions[i][2]    for i in range(len(solutions))]),
            ("Total time"    , [solutions[i][3][0] for i in range(len(solutions))]),
            ("Total restarts", [sum([solutions[i][4][j]["restarts"] for j in range(len(solutions[i][4]))]) 
                                                   for i in range(len(solutions))]),
            ("Total conflicts", [sum([solutions[i][4][j]["conflicts"] for j in range(len(solutions[i][4]))]) 
                                                   for i in range(len(solutions))]),
            ("Total decisions", [sum([solutions[i][4][j]["decisions"] for j in range(len(solutions[i][4]))]) 
                                                   for i in range(len(solutions))]),
            ("Total propagations", [sum([solutions[i][4][j]["propagations"] for j in range(len(solutions[i][4]))]) 
                                                   for i in range(len(solutions))]),
           ] + concat_all([
            [(f"Time[{j + 1}]", [solutions[i][3][1][j] if j < len(solutions[i][3][1]) else "-"
                                                   for i in range(len(solutions))]),
             (f"N_wanted[{j + 1}]", [solutions[i][4][j]["n_wanted"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
             (f"N_found[{j + 1}]", [solutions[i][4][j]["n_found"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
             (f"Restarts[{j + 1}]", [solutions[i][4][j]["restarts"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
             (f"Conflicts[{j + 1}]", [solutions[i][4][j]["conflicts"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
             (f"Decisions[{j + 1}]", [solutions[i][4][j]["decisions"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
             (f"Propagations[{j + 1}]", [solutions[i][4][j]["propagations"] if j < len(solutions[i][4]) else "-"
                                                   for i in range(len(solutions))]),
            ] for j in range(max_it)])
    dictionary = {item[0]: item[1] for item in sets}
    return pd.DataFrame(dictionary)

In [45]:
rand.seed(0)

strategies = [
    ("naive_bottomup", naive_bottomup_strategy), 
    ("naive_topdown", naive_topdown_strategy), 
    ("assumptions", assumptions_strategy), 
    ("single_assumption", single_assumption_strategy), 
    ("clauses", clauses_strategy),
    ("core", core_strategy),
]

solvers = [
    ("Cadical", Cadical), 
    #("Glucose3", Glucose3), 
    ("Glucose4", Glucose4), 
    #("Lingeling", Lingeling), 
    #("MapleChrono", MapleChrono), 
    #("MapleCM", MapleCM), 
    #("Maplesat", Maplesat), 
    #("Minicard", Minicard), 
    ("Minisat22", Minisat22), 
    #("MinisatGH", MinisatGH),
]

graphs = read_graphs()[7:8]#[(file, graph) for file, graph in read_graphs() if file == 'myciel5.col.txt']#
#print(graphs)
#graphs = [("sample1", generate_graph(25, 100))]
#graphs = [("sample0", (5, [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (2, 3), (3, 4), (4, 1)]))]

In [46]:
solutions = list()
for graph_name, (n, edges) in graphs: 
    for solver_name, solver in solvers:
        for strategy_name, strategy in strategies:
            colors, times, stats = strategy(n, edges, solver)
            #super_naive_strategy(n, edges, solver)
            solutions.append((graph_name, solver_name, strategy_name, times, stats))
            print_results(strategy_name, solver_name, graph_name, colors, times)

solutions2csv(solutions).to_csv("solutions_3.csv", index=None, encoding="utf-8")

strategy=naive_bottomup solver=Cadical graph=huck.col.txt time=10.240315513998212:
SAT
(11, {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 2, 9: 1, 10: 2, 11: 0, 12: 3, 13: 1, 14: 0, 15: 0, 16: 1, 17: 3, 18: 0, 19: 2, 20: 3, 21: 4, 22: 4, 23: 0, 24: 4, 25: 0, 26: 2, 27: 0, 28: 5, 29: 0, 30: 4, 31: 0, 32: 0, 33: 5, 34: 0, 35: 0, 36: 0, 37: 5, 38: 6, 39: 6, 40: 3, 41: 1, 42: 2, 43: 6, 44: 0, 45: 2, 46: 4, 47: 1, 48: 7, 49: 8, 50: 1, 51: 7, 52: 1, 53: 0, 54: 9, 55: 4, 56: 5, 57: 2, 58: 10, 59: 6, 60: 0, 61: 6, 62: 2, 63: 7, 64: 1, 65: 0, 66: 7, 67: 7, 68: 3, 69: 1, 70: 2, 71: 3, 72: 8, 73: 4})


strategy=naive_topdown solver=Cadical graph=huck.col.txt time=7.047467788001086:
SAT
(11, {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 2, 9: 1, 10: 2, 11: 0, 12: 3, 13: 1, 14: 0, 15: 0, 16: 1, 17: 3, 18: 0, 19: 2, 20: 3, 21: 4, 22: 4, 23: 0, 24: 4, 25: 0, 26: 2, 27: 0, 28: 5, 29: 0, 30: 4, 31: 0, 32: 0, 33: 5, 34: 0, 35: 0, 36: 0, 37: 5, 38: 6, 39: 6, 40: 3, 41: 1, 42: 2, 43: 6, 44: 0,

strategy=assumptions solver=Minisat22 graph=huck.col.txt time=7182.277194363998:
SAT
(11, {0: 10, 1: 5, 2: 0, 3: 1, 4: 5, 5: 3, 6: 0, 7: 9, 8: 5, 9: 0, 10: 1, 11: 5, 12: 6, 13: 3, 14: 1, 15: 6, 16: 5, 17: 0, 18: 10, 19: 6, 20: 8, 21: 3, 22: 0, 23: 1, 24: 9, 25: 4, 26: 5, 27: 0, 28: 7, 29: 9, 30: 7, 31: 2, 32: 2, 33: 7, 34: 7, 35: 0, 36: 1, 37: 6, 38: 9, 39: 8, 40: 8, 41: 0, 42: 2, 43: 1, 44: 2, 45: 0, 46: 0, 47: 1, 48: 4, 49: 0, 50: 0, 51: 4, 52: 3, 53: 6, 54: 2, 55: 0, 56: 6, 57: 0, 58: 3, 59: 4, 60: 0, 61: 4, 62: 4, 63: 1, 64: 10, 65: 6, 66: 3, 67: 9, 68: 0, 69: 9, 70: 10, 71: 7, 72: 7, 73: 1})


strategy=single_assumption solver=Minisat22 graph=huck.col.txt time=7146.387937080999:
SAT
(11, {0: 10, 1: 5, 2: 0, 3: 1, 4: 5, 5: 3, 6: 0, 7: 9, 8: 5, 9: 0, 10: 1, 11: 5, 12: 6, 13: 3, 14: 1, 15: 6, 16: 5, 17: 0, 18: 10, 19: 6, 20: 8, 21: 3, 22: 0, 23: 1, 24: 9, 25: 4, 26: 5, 27: 0, 28: 7, 29: 9, 30: 7, 31: 2, 32: 2, 33: 7, 34: 7, 35: 0, 36: 1, 37: 6, 38: 9, 39: 8, 40: 8, 41: 0, 42: 2, 43: 